<a href="https://colab.research.google.com/github/chiruthejaswi/Task1/blob/main/task1_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Network from Scratch: Income Classification Report**

**Objective**

Predict whether a person earns more than $50,000 per year using a feedforward neural network built from scratch in PyTorch on the UCI Adult Income dataset. This problem involves hanling tabular data with both categorical and numerical features, making it an ideal candidate for deep learning experimentation and ablation studies.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder,MinMaxScaler,StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset
from sklearn.metrics import accuracy_score,f1_score
import numpy as np
from torch.optim import SGD,Adam,RMSprop
import matplotlib.pyplot as plt
import seaborn as sns
!pip install shap
import shap
shap.initjs()


# **Loading Data:**

We begin by loading the Adult Income dataset. It comes in two parts: adult.data (train) and adult.test (test). Both are loaded with consistent column names, and the test file skips the first line as it contains comments.

We then combine both datasets into a single DataFrame for uniform preprocessing.


here is the link for the data set:https://drive.google.com/drive/folders/1nW-dv41X-xHxz3qrh8-YnweRSp-lKG4I?usp=drive_link


In [ ]:
# load dataset and naming columns
column_names=['age','workclass','fnlwgt','education','education-num','marital-status',
              'occupation','relationship','race','sex','capital-gain','capital-loss',
              'hours-per-week','native-country','income']

# loading train and test datasets
df_train =pd.read_csv("/content/drive/MyDrive/adult_income_dataset/adult.data",header=None,names=column_names,na_values='?')
df_test =pd.read_csv("/content/drive/MyDrive/adult_income_dataset/adult.test",names=column_names,na_values='?',skiprows=1) # skipping one row because we have a comment line at its head

# combining both for uniform preprocessing
df=pd.concat([df_train,df_test])
df.head()

# **Handling Missing Values**

We identify and handle missing values using two strategies:

* **Dropping Missing Values:**

We use df.dropna() to remove rows with missing values. This leads to less data but avoids assumptions during imputation.

* **Imputing Missing Values:**


We apply imputation

***For categorical columns:*** fill missing values with the most frequent (mode).

***For numerical columns:*** fill missing values with the mean.

This retains all rows in the dataset while providing reasonable estimates for missing entries.

We now have two versions of the dataset:

df_dropna: rows with missing values dropped

df_imputed: missing values imputed

In [ ]:
# Handling missing values

# Let's chcek the missing values first
df.isna().sum()

# 1.by dropping missing rows
df_dropna=df.dropna()

# 2.by imputing missing
df_imputed = df.copy()
# selecting categorical columns and numerical columns seperately
cat_columns = df_imputed.select_dtypes(include='object').columns
num_columns = df_imputed.select_dtypes(include='number').columns
# imputed the categorical values with mode and numerical values with mean
imputed_cat = SimpleImputer(strategy='most_frequent')
imputed_num = SimpleImputer(strategy='mean')

df_imputed[cat_columns] = imputed_cat.fit_transform(df_imputed[cat_columns])
df_imputed[num_columns] = imputed_num.fit_transform(df_imputed[num_columns])
df_imputed.head()

# **Encoding Categorical Features**

Categorical data must be encoded to numerical form for most ML models. We apply three different encoding techniques to both df_imputed and df_dropna.

# Label Encoding
Each unique category is converted to a unique integer. Suitable for tree-based models but may impose unintended order.

# Ordinal Encoding
Like label encoding, but applied with OrdinalEncoder() across all columns in one go.

# One-Hot Encoding
Each category becomes a separate binary column. This avoids false ordinal relationships but increases dimensionality.

* We also ensure the target column income is excluded from encoding and handled separately.

This results in six encoded datasets:

{df_label_encoded1, df_ordinal_encoded1, df_onehot_encoded1 (from df_imputed)

df_label_encoded2, df_ordinal_encoded2, df_onehot_encoded2 (from df_dropna)}

**combination of imputed dataframe with OneHotEncoding,Ordinal Encoding,Label Encoding**

In [ ]:
# define categorical and numerical columns
cat_columns= df_imputed.select_dtypes(include='object').columns
num_columns =df_imputed.select_dtypes(include='number').columns

# Label Encoding
df_label_encoded1=df_imputed.copy()
label_encoders={}

for col in cat_columns:
    le = LabelEncoder()
    df_label_encoded1[col]= le.fit_transform(df_label_encoded1[col])
    label_encoders[col] =le
print("Label Encoding completed. Shape:", df_label_encoded1.shape)

# Ordinal Encoding
df_ordinal_encoded1= df_imputed.copy()
ordinal_encoder= OrdinalEncoder()
df_ordinal_encoded1[cat_columns]= ordinal_encoder.fit_transform(df_ordinal_encoded1[cat_columns])

print("Ordinal Encoding completed.Shape:", df_ordinal_encoded1.shape)

onehot_encoder= OneHotEncoder(sparse_output=False, handle_unknown='ignore')
onehot_encoded= onehot_encoder.fit_transform(df_imputed[cat_columns])
onehot_feature_names =onehot_encoder.get_feature_names_out(cat_columns)

# Combine with numerical columns
df_onehot= pd.DataFrame(onehot_encoded, columns=onehot_feature_names)
df_onehot_encoded1 =pd.concat([df_imputed[num_columns].reset_index(drop=True), df_onehot], axis=1)

print("One-Hot Encoding done.Shape:",df_onehot_encoded1.shape)

**Combination of dropped data frame with OneHotEncoding,Ordinal encoding,Label Encoding**

In [ ]:
# Label Encoding
df_label_encoded2=df_dropna.copy()
label_encoders={}

for col in cat_columns:
    le = LabelEncoder()
    df_label_encoded2[col]= le.fit_transform(df_label_encoded2[col])
    label_encoders[col] =le
print("Label Encoding done. Shape:", df_label_encoded2.shape)

# Ordinal Encoding
df_ordinal_encoded2 = df_dropna.copy()
ordinal_encoder = OrdinalEncoder()
df_ordinal_encoded2[cat_columns] = ordinal_encoder.fit_transform(df_ordinal_encoded2[cat_columns])

print("Ordinal Encoding done. Shape:", df_ordinal_encoded2.shape)

onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
onehot_encoded = onehot_encoder.fit_transform(df_dropna[cat_columns])
onehot_feature_names = onehot_encoder.get_feature_names_out(cat_columns)

# Combine with numerical columns
df_onehot = pd.DataFrame(onehot_encoded, columns=onehot_feature_names)
df_onehot_encoded2 = pd.concat([df_dropna[num_columns].reset_index(drop=True), df_onehot], axis=1)

print("One-Hot Encoding done. Shape:", df_onehot_encoded2.shape)

# **Feature Scaling**

To bring numerical features into similar ranges and improve model convergence, we apply three different scalers:

**MinMaxScaler:** Scales to [0, 1] range.

**StandardScaler:** Standardizes with mean = 0 and std = 1.

**RobustScaler:** Uses median and IQR; better for outliers.

We apply each scaler to each of the 6 encoded datasets, resulting in 18 total scaled feature sets.

These are stored in a dictionary scaled_dfs.

In [ ]:
# list of scalers
scalers= {
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()}

# list of data frames
encoded_dfs= {
    'label_imputed': df_label_encoded1,
    'label_dropna': df_label_encoded2,
    'ordinal_imputed': df_ordinal_encoded1,
    'ordinal_dropna': df_ordinal_encoded2,
    'onehot_imputed': df_onehot_encoded1,
    'onehot_dropna': df_onehot_encoded2}
# Dictionary to store final scaled dataframes
scaled_dfs={}

# applying scalers for each encoded data frame
for name,df in encoded_dfs.items():
    # Identify numeric columns
    numeric_cols=df.select_dtypes(include='number').columns

    for scaler_name,scaler in scalers.items():
        df_copy= df.copy()
        df_copy[numeric_cols] =scaler.fit_transform(df_copy[numeric_cols])

        combo_name= f"{name}_{scaler_name}"
        scaled_dfs[combo_name]= df_copy
        print(f"{combo_name} done.Shape: {df_copy.shape}")

# **Train/Validation/Test split**
For each of the 18 scaled datasets, we perform a 70/15/15 split:

Split into 70% train and 30% temp.

Split the temp set equally into validation and test (15% each).

This ensures robust model evaluation.

We handle the target column differently based on encoding:

For label/ordinal encoded datasets, y = df['income']

For one-hot encoded datasets, y = df[['income_ <=50K', 'income_ >50K']] and all income columns are excluded from X.

The splits are stored in a dictionary splits

In [ ]:
# Dictionary to store splits
splits={}

for name,df in scaled_dfs.items():
    if 'onehot' in name:
        # For one-hot encoded dataframes, drop all 'income' related columns
        income_cols=[col for col in df.columns if 'income' in col]
        X= df.drop(columns=income_cols)
        # Assuming the last two income columns are the target for one-hot encoded data
        y = df[['income_ <=50K', 'income_ >50K']]
    else:
        # For Label and Ordinal encoded dataframes, drop the 'income' column
        X = df.drop('income', axis=1)
        y = df['income']

    # First split: Train (70%) and Temp (30%)
    X_train,X_temp,y_train,y_temp =train_test_split(X,y,test_size=0.30,random_state=42,stratify=y if not isinstance(y,pd.DataFrame) else y.iloc[:,0])

    # Second split: Validation (15%) and Test (15%) from the 30%
    X_val,X_test,y_val,y_test=train_test_split(X_temp, y_temp,test_size=0.50,random_state=42,stratify=y_temp if not isinstance(y_temp, pd.DataFrame) else y_temp.iloc[:, 0])


    splits[name] = {
        'X_train': X_train,
        'y_train': y_train,
        'X_val': X_val,
        'y_val': y_val,
        'X_test': X_test,
        'y_test': y_test}

    print(f"Split done for {name} — Train:{X_train.shape},Val:{X_val.shape},Test:{X_test.shape}")

Now that our dataset has been fully preprocessed —> encoded, scaled, and
split.

In this section, we'll implement a feedforward neural network using PyTorch.Our goal is to classify whether a person earns more than $50K per year based on a range of demographic and work-related features.

# **2.Model Building and Architecture Ablation**

This section investigates the effect of varying the network architecture (depth, width), activation functions, and the inclusion of regularization layers like dropout and batch normalization. All models were trained and evaluated across all 18 dataset preprocessing combinations to measure the robustness and generalization power of each architecture.

**Objective**

To explore how the structure of a feedforward neural network affects classification performance. Specifically, we test:

* Shallow vs deep networks

* ReLU vs Tanh vs LeakyReLU

* Dropout for regularization

* BatchNorm for training stability

**Model Implementation**

We implemented a modular class FeedforwardNN with the following properties:

* Accepts arbitrary number of hidden layers (hidden_dims)

* Activations are passed as constructor arguments (nn.ReLU, nn.Tanh, nn.LeakyReLU)

* Optional dropout and batch normalization

* Output layer is always nn.Linear → Sigmoid (for BCELoss)

Each model was trained using:

* Optimizer: Adam (lr = 0.001)

* Loss: BCELoss

* Batch size: 64

* Early stopping with patience = 3

* Max epochs: 30

* Trained on all 18 preprocessing pipelines

**Training Strategy**

The training was done using a train_model() function with early stopping that monitored F1 score on validation set. For each dataset-architecture pair:

* Training loss was tracked over epochs

* Best model (based on F1) was saved and evaluated on the validation set

**Results Recorded**

For each dataset + architecture combination:

* Final validation accuracy

* Final validation F1 score

* Per-epoch loss (stored in loss_curves)

* Saved into arch_results_df → exported as arch_results.csv

 **Observations**

* Model A4 (deep, ReLU + Dropout + BatchNorm) consistently performed best across most datasets

* Tanh activation worked well with dropout but converged slower

* LeakyReLU + BatchNorm was stable but didn't outperform A4

* Shallow networks (A1, A3) had faster convergence but poorer generalization



**Conclusion**

* A4 ([128, 64, 32], ReLU, Dropout, BatchNorm) was selected as the best architecture

* This architecture was used as the fixed baseline for Sections 3–6

* Depth + normalization + dropout → best generalization performance

In [ ]:
#  Define a customizable Feedforward Neural Network
class FeedforwardNN(nn.Module):
    def __init__(self,input_dim,hidden_dims,activation_fn,use_dropout=False,use_batchnorm=False):
        super(FeedforwardNN,self).__init__()
        layers= []
        in_dim=input_dim

        # Loop over each hidden layer dimension
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(in_dim,hidden_dim))  # Fully connected layer

            # Add batch normalization if enabled (helps training stability and speed)
            if use_batchnorm:
                layers.append(nn.BatchNorm1d(hidden_dim))

            # Add activation (e.g., ReLU, Tanh, LeakyReLU)
            layers.append(activation_fn())

            # Add dropout if enabled (helps regularization, prevents overfitting)
            if use_dropout:
                layers.append(nn.Dropout(0.3))  # Fixed dropout rate for consistency

            in_dim=hidden_dim  # Output of current layer becomes input to next

        # Final output layer: 1 neuron with Sigmoid for binary classification
        layers.append(nn.Linear(in_dim,1))
        layers.append(nn.Sigmoid())

        self.model=nn.Sequential(*layers)  # Combine all layers into one sequential block

    def forward(self,x):
        return self.model(x)  # Forward pass through the model
# Architecture configurations to experiment with
# Each config varies in depth, activation function, dropout, and batch normalization
configs = [
    {"name": "2 layers (64, 32) - ReLU", "hidden_dims": [64, 32], "activation": nn.ReLU, "dropout": False, "batchnorm": False},
    {"name": "3 layers (128, 64, 32) - Tanh+Dropout", "hidden_dims": [128, 64, 32], "activation": nn.Tanh, "dropout": True, "batchnorm": False},
    {"name": "2 layers (64, 32) -LeakyReLU+BatchNorm", "hidden_dims": [64, 32], "activation": nn.LeakyReLU, "dropout": False, "batchnorm": True},
    {"name": "3 layers (128, 64, 32)-ReLU+ Dropout + BatchNorm", "hidden_dims": [128, 64, 32], "activation": nn.ReLU, "dropout": True, "batchnorm": True}
]

device= torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
arch_results= []     # To store (dataset, architecture, acc, f1)
loss_curves= {}      # To track training loss per epoch

# Function to evaluate a trained model on validation data
def evaluate_model(model, X, y, device='cpu'):
    model.eval()  # Set model to evaluation mode (disables dropout/batchnorm updates)
    with torch.no_grad(): # No gradients needed for evaluation
        X = X.to(device).float()

        # If labels are in DataFrame/Series format, convert to binary (0/1)
        if isinstance(y, pd.Series) or isinstance(y, pd.DataFrame):
            y = y.apply(lambda x: 1 if '>50K' in str(x) else 0).values
        y = torch.tensor(y).to(device).int()

        preds = model(X).view(-1)  # Flatten output
        preds_cls = (preds > 0.5).int()  # Convert probabilities to 0/1

        # Compute accuracy and F1 score
        acc = accuracy_score(y.cpu(), preds_cls.cpu())
        f1 = f1_score(y.cpu(), preds_cls.cpu())
        return acc, f1

# Training function with early stopping based on F1 score
def train_model(model, train_loader, val_data, optimizer, criterion, patience=3, max_epochs=30):
    X_val, y_val = val_data
    best_f1 = 0               # Best F1 score seen so far
    early_stop_count = 0      # How many epochs since improvement
    best_state = None         # To store best model weights
    losses = []               # Track training loss per epoch

    for epoch in range(max_epochs):
        model.train()
        total_loss = 0

        # Iterate over mini-batches from training data
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device).view(-1, 1).float()

            optimizer.zero_grad()          # Clear gradients
            out = model(xb).view(-1, 1)    # Forward pass
            loss = criterion(out, yb)      # Compute loss
            loss.backward()                # Backprop
            optimizer.step()              # Update weights
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        losses.append(avg_loss)

        # Evaluate on validation set after each epoch
        acc, f1 = evaluate_model(model, X_val, y_val, device=device)
        print(f"Epoch {epoch+1:02d} - Loss: {avg_loss:.4f} | Val Acc: {acc:.4f} | F1: {f1:.4f}")

        # Early stopping check
        if f1 > best_f1:
            best_f1 = f1
            best_state = model.state_dict()
            early_stop_count = 0  # reset counter if improved
        else:
            early_stop_count += 1
            if early_stop_count >= patience:
                print("Early stopping.")
                break

    # Restore best model state
    if best_state:
        model.load_state_dict(best_state)

    return model, losses
# Run all models on all 18 preprocessed datasets
for dataset_name, split in splits.items():
    print(f"\n Dataset: {dataset_name}")

    # Ensure label is Series, not DataFrame
    y_train_series = split['y_train']
    y_val_series = split['y_val']
    if isinstance(y_train_series, pd.DataFrame):
        y_train_series = y_train_series.iloc[:, 0]
    if isinstance(y_val_series, pd.DataFrame):
        y_val_series = y_val_series.iloc[:, 0]

    # Convert data to PyTorch tensors
    X_train= torch.tensor(split['X_train'].values).float()
    y_train= torch.tensor(y_train_series.apply(lambda x: 1 if '>50K' in str(x) else 0).values).float()
    X_val = torch.tensor(split['X_val'].values).float()
    y_val = torch.tensor(y_val_series.apply(lambda x: 1 if '>50K' in str(x) else 0).values).int()

    # Use DataLoader for mini-batch training
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)

    # Try each architecture configuration
    for cfg in configs:
        print(f"\n Model: {cfg['name']}")

        # Instantiate the model with the given configuration
        model = FeedforwardNN(
            input_dim=X_train.shape[1],
            hidden_dims=cfg['hidden_dims'],
            activation_fn=cfg['activation'],
            use_dropout=cfg['dropout'],
            use_batchnorm=cfg['batchnorm']
        ).to(device)

        # Set optimizer and loss
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.BCELoss()

        # Train the model and evaluate
        model, losses = train_model(model, train_loader, (X_val, y_val), optimizer, criterion)
        acc, f1 = evaluate_model(model, X_val, y_val, device=device)

        print(f" Final Val Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")

        # Save results
        key = f"{dataset_name} | {cfg['name']}"
        arch_results.append((dataset_name, cfg['name'], acc, f1))
        loss_curves[key] = losses
# Save the results to a CSV file for analysis and visualization
arch_results_df = pd.DataFrame(arch_results, columns=["Dataset", "Architecture", "Val Accuracy", "F1 Score"])
arch_results_df.to_csv("arch_results.csv", index=False)
print("Saved to arch_results.csv")


In [ ]:
# Example: visualize a few losses
plt.figure(figsize=(12, 6))
for name,losses in list(loss_curves.items())[:5]:  # plot first 5 configs
    plt.plot(losses,label=name)
plt.title("Training Loss Curves (first 5)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(fontsize="small")
plt.tight_layout()
plt.savefig("loss_curves_section2.png")
plt.show()


# **3.Loss Function and Optimizer Ablation**

This section explores the impact of different loss functions, optimizers, and learning rates on training dynamics and model generalization. Understanding how these training hyperparameters interact is crucial for achieving fast convergence and high performance on validation data.



**Objective**

To identify the best combination of:

* Loss Function:

    1.BCELoss (requires Sigmoid in the model)

    2.BCEWithLogitsLoss (more stable; no Sigmoid in model)

* Optimizer:

    * Adam (adaptive)

    * SGD (vanilla)

    * RMSprop (adaptive)

* Learning Rate:

    * 0.001 (safe)

    * 0.01 (moderate)

    * 0.1 (aggressive)



**Experimental Setup**

* Architecture: Fixed as best from Section 2 → [128, 64, 32], ReLU, Dropout (0.3), BatchNorm

* Dataset: Fixed as best preprocessed version → label_imputed_StandardScaler

* Training:

  * Optimizer and loss varied as per combinations

  * Early stopping used (patience = 3)

  * Epochs: max 30

  * Batch Size: 64

For BCEWithLogitsLoss, the final Sigmoid in the model is replaced with nn.Identity()
Now we have total 18 experimental setups


**Observations**

* BCEWithLogitsLoss consistently outperformed BCELoss, likely due to improved numerical stability.

* Adam was the most robust optimizer, performing well at all learning rates, especially 0.001.

* SGD was sensitive to learning rate. At 0.01 it worked decently, but diverged at 0.1.

* RMSprop performed better than SGD, but not better than Adam.

* High learning rate (0.1) caused unstable training for SGD and RMSprop.

**Conclusion**

* Best combination : BCEWithLogitsLoss + Adam + lr = 0.001

* This combination was used in later sections (regularization, tuning, explainability)

* BCEWithLogitsLoss is recommended when using logits directly (no Sigmoid in model)

* Adam is a good default optimizer; it adapts well and avoids manual tuning pitfalls



In [ ]:
# Fixed config
input_dim= splits['label_imputed_StandardScaler']['X_train'].shape[1]
hidden_dims= [128, 64, 32]
activation_fn= nn.ReLU
use_dropout= True
use_batchnorm= True
batch_size= 64
patience= 3
max_epochs= 30

# Combinations to test
loss_functions= {
    "BCELoss": nn.BCELoss(),
    "BCEWithLogitsLoss": nn.BCEWithLogitsLoss()
}

optimizers= ["Adam","SGD","RMSprop"]
learning_rates= [0.001,0.01,0.1]

# Prepare data (from best dataset)
split= splits['label_imputed_StandardScaler']
X_train= torch.tensor(split['X_train'].values).float()
X_val =torch.tensor(split['X_val'].values).float()

y_train_series= split['y_train']
y_val_series= split['y_val']
if isinstance(y_train_series,pd.DataFrame):
    y_train_series= y_train_series.iloc[:, 0]
if isinstance(y_val_series, pd.DataFrame):
    y_val_series= y_val_series.iloc[:, 0]

y_train= torch.tensor(y_train_series.apply(lambda x: 1 if x > 0 else 0).values).float()
y_val= torch.tensor(y_val_series.apply(lambda x: 1 if x > 0 else 0).values).int()

train_loader =DataLoader(TensorDataset(X_train,y_train),batch_size=batch_size,shuffle=True)

# Results store
opt_results=[]

# Loop through combinations
for loss_name,loss_fn in loss_functions.items():
    for opt_name in optimizers:
        for lr in learning_rates:
            print(f"\n {loss_name} + {opt_name} + lr={lr}")

            model = FeedforwardNN(
                input_dim=input_dim,
                hidden_dims=hidden_dims,
                activation_fn=activation_fn,
                use_dropout=use_dropout,
                use_batchnorm=use_batchnorm
            ).to(device)

            # Choose optimizer
            if opt_name== "Adam":
                optimizer = optim.Adam(model.parameters(),lr=lr)
            elif opt_name== "SGD":
                optimizer = optim.SGD(model.parameters(),lr=lr)
            elif opt_name== "RMSprop":
                optimizer = optim.RMSprop(model.parameters(),lr=lr)

            # Remove final Sigmoid if using BCEWithLogitsLoss
            if isinstance(loss_fn,nn.BCEWithLogitsLoss):
                model.model[-1]= nn.Identity()

            # Train
            model,_ =train_model(model, train_loader, (X_val, y_val), optimizer, loss_fn, patience, max_epochs)

            # Eval
            acc,f1 =evaluate_model(model,X_val,y_val,device=device)
            print(f"Accuracy: {acc:.4f}, F1: {f1:.4f}")
            opt_results.append((loss_name,opt_name,lr,acc,f1))

# Convert to DataFrame
opt_results_df= pd.DataFrame(opt_results, columns=["Loss","Optimizer","Learning Rate","Val Accuracy","F1 Score"])
opt_results_df.to_csv("optimizer_ablation.csv",index=False)
print("Saved to optimizer_ablation.csv")


In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(data=opt_results_df,x="Optimizer",y="F1 Score",hue="Loss")
plt.title("F1 Score per Loss/Optimizer Combo (Best Dataset)")
plt.tight_layout()
plt.savefig("optimizer_ablation_plot.png")
plt.show()


# **4.Regularization & Overfitting Control**

This section investigates the effect of dropout and weight decay (L2 regularization) on the model’s ability to generalize and mitigate overfitting. The objective is to compare model performance across various combinations of these regularization strategies.



**Objective**

To study how regularization affects overfitting and generalization. Specifically:

* Dropout rates tested: 0.0, 0.2, 0.5, 0.7

* Weight decay values tested: 0.0, 1e-4, 1e-2

* We also used early stopping with patience = 3 for all models



**Experimental Setup**

* Model architecture: [128, 64, 32], ReLU activations, BatchNorm enabled

* Loss: BCEWithLogitsLoss

* Optimizer: Adam with varying weight_decay

* Dropout: controlled via nn.Dropout(p) at each layer

* Dataset: label_imputed_StandardScaler

* Batch size: 64

* Max epochs: 30

* Early stopping: Stops if F1 score doesn't improve for 3 epochs

**Observations**

* Moderate dropout (0.2) with low/no weight decay gave the best overall results among all tested configs.

* A small weight decay (1e-2) sometimes helped (e.g., with no dropout) but too much regularization hurt performance.

* High dropout (0.7) consistently reduced F1 score, likely due to underfitting.

* Early stopping successfully prevented overfitting across all runs, with most training stopping within 5–13 epochs.

* Best configuration: Dropout = 0.0, Weight Decay = 1e-2 → F1 = 0.5031

**Conclusion**

* Dropout and weight decay are useful but must be balanced — too much causes underfitting.

* Best validation performance achieved with either:

  * Dropout 0.0 + Weight Decay 1e-2, or

  * Dropout 0.2 + Weight Decay 1e-4

* Early stopping is a very effective safeguard against overfitting and should be standard practice.

In [ ]:
# Regularization hyperparameters to sweep
dropout_rates = [0.0,0.2,0.5,0.7]
weight_decays = [0.0,1e-4,1e-2]

# Get best dataset
split= splits['label_imputed_StandardScaler']
X_train= torch.tensor(split['X_train'].values).float()
X_val= torch.tensor(split['X_val'].values).float()

y_train_series =split['y_train']
y_val_series= split['y_val']
if isinstance(y_train_series,pd.DataFrame):
    y_train_series= y_train_series.iloc[:, 0]
if isinstance(y_val_series,pd.DataFrame):
    y_val_series= y_val_series.iloc[:,0]

y_train= torch.tensor(y_train_series.apply(lambda x: 1 if x > 0 else 0).values).float()
y_val =torch.tensor(y_val_series.apply(lambda x: 1 if x > 0 else 0).values).int()

train_loader= DataLoader(TensorDataset(X_train, y_train),batch_size=64,shuffle=True)

# Store results
reg_results= []

# Loop over all combinations
for drop in dropout_rates:
    for wd in weight_decays:
        print(f"\n Dropout: {drop}, Weight Decay: {wd}")

        model= FeedforwardNN(
            input_dim=X_train.shape[1],
            hidden_dims=[128, 64, 32],
            activation_fn=nn.ReLU,
            use_dropout=True if drop > 0 else False,
            use_batchnorm=True
        ).to(device)

        # Manually set dropout rate (if needed)
        if drop > 0:
            for layer in model.model:
                if isinstance(layer, nn.Dropout):
                    layer.p = drop

        # Remove final sigmoid for BCEWithLogitsLoss
        model.model[-1]= nn.Identity()

        optimizer= optim.Adam(model.parameters(), lr=0.001, weight_decay=wd)
        criterion= nn.BCEWithLogitsLoss()

        model, _= train_model(model, train_loader, (X_val, y_val), optimizer, criterion, patience=3, max_epochs=30)
        acc, f1 = evaluate_model(model, X_val, y_val, device=device)

        print(f"Val Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")
        reg_results.append((drop,wd,acc,f1))

# Save results
reg_results_df= pd.DataFrame(reg_results, columns=["Dropout", "Weight Decay", "Val Accuracy", "F1 Score"])
reg_results_df.to_csv("regularization_ablation.csv", index=False)
print("Saved to regularization_ablation.csv")


In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=reg_results_df, x="Dropout", y="F1 Score",hue="Weight Decay", marker="o")
plt.title("F1 Score vs Dropout Rate for Different Weight Decays")
plt.savefig("reg_ablation_plot.png")
plt.show()


# **5.Model Evaluation**

In this section, we conduct a thorough evaluation of the model's performance using multiple classification metrics and assess potential bias across sensitive features like sex and race.

**Objective**

* Evaluate trained models across all 18 preprocessed datasets

* Compute:

  * Accuracy

  * Precision

  * Recall

  * F1-score

* Generate and interpret confusion matrices

* Analyze group-wise performance across sex and race

* Identify patterns of bias or unfairness

**Results Summary**

Most models achieved high overall accuracy,F1-scores on the best-performing datasets. Precision was generally higher than recall, indicating that the model is more conservative when predicting high-income labels. Some models performed slightly worse on datasets with aggressive preprocessing or imbalanced encoding.



**Confusion Matrix Interpretation**

The confusion matrix shows how often the model predicted correctly versus where it made mistakes:

* The top-left cell shows the number of true negatives (correctly predicted <=50K).

* The bottom-right cell shows the true positives (correctly predicted >50K).

* The off-diagonal cells show misclassifications:

* False positives: predicted >50K when it was actually <=50K.

* False negatives: predicted <=50K when it was actually >50K.

Most models had more false negatives than false positives, meaning the model tends to miss people who earn >50K, which is a recall issue.

**Bias Evaluation**

To check for bias, we measured accuracy separately for different groups:

Sex-based analysis showed that the model consistently performed better for males than females. The difference in accuracy between the groups was often more than 5%.

Race-based analysis revealed that the model had the highest accuracy for white individuals and lower accuracy for Black and Asian groups. This gap sometimes exceeded 10–15%, which raises fairness concerns.

These patterns were consistent across most preprocessing variants, meaning that the bias was not caused by a single encoding or scaling method

**Conclusion**

* The model performs well on average, but there are significant differences in accuracy across demographic groups.

* These disparities suggest potential bias in the model, possibly due to imbalance in the data.

* While high-level metrics like accuracy and F1 look good, a closer inspection reveals unequal performance across subgroups, which is a critical concern in real-world deployment.

* These findings highlight the importance of evaluating fairness and not relying solely on aggregate performance metrics.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report,confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load full raw dataset to recover sensitive features
raw= fetch_openml(name='adult',version=2,as_frame=True)
data= raw['data']
target= raw['target']

# Best model config
def get_best_model(input_dim):
    model= FeedforwardNN(
        input_dim=input_dim,
        hidden_dims=[128, 64, 32],
        activation_fn=nn.ReLU,
        use_dropout=True,
        use_batchnorm=True
    ).to(device)
    model.model[-1] = nn.Identity()  # For BCEWithLogitsLoss
    return model

# Storage
eval_results= []
bias_results= []
confusion_matrices= {}

# Loop over all dataset splits
for dataset_name, split in splits.items():
    print(f"\n Evaluating dataset: {dataset_name}")

    # Prepare full train set (train + val)
    X_train= pd.concat([split['X_train'], split['X_val']])
    y_train= pd.concat([split['y_train'], split['y_val']])
    X_test =split['X_test']
    y_test= split['y_test']

    # Handle Series/DataFrame
    if isinstance(y_train,pd.DataFrame):
        y_train= y_train.iloc[:,0]
    if isinstance(y_test,pd.DataFrame):
        y_test= y_test.iloc[:,0]

    # Convert to tensors
    X_train_tensor= torch.tensor(X_train.values).float()
    y_train_tensor =torch.tensor(y_train.apply(lambda x: 1 if x > 0 else 0).values).float()
    X_test_tensor =torch.tensor(X_test.values).float()
    y_test_tensor= torch.tensor(y_test.apply(lambda x: 1 if x > 0 else 0).values).int()


    train_loader= DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)

    # Train model
    model= get_best_model(X_train.shape[1])
    optimizer =optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-2)
    criterion= nn.BCEWithLogitsLoss()
    model, _ =train_model(model, train_loader, (X_test_tensor, y_test_tensor), optimizer, criterion)

    # Predict
    model.eval()
    with torch.no_grad():
        logits= model(X_test_tensor.to(device)).view(-1)
        preds= (logits > 0).int().cpu()
        y_true =y_test_tensor.cpu()

    # Metrics
    acc= accuracy_score(y_true, preds)
    prec= precision_score(y_true, preds, zero_division=0)
    rec= recall_score(y_true, preds)
    f1 =f1_score(y_true, preds)

    eval_results.append((dataset_name,acc,prec,rec,f1))

    # Confusion matrix
    # Check if there is more than one unique class in true and predicted labels before plotting
    if len(np.unique(y_true)) > 1 and len(np.unique(preds)) > 1:
        cm = confusion_matrix(y_true, preds)
        confusion_matrices[dataset_name] = cm
        disp= ConfusionMatrixDisplay(cm,display_labels=["<=50K",">50K"])
        disp.plot()
        plt.title(f"Confusion Matrix:{dataset_name}")
        plt.savefig(f"confusion_matrix_{dataset_name}.png")
        plt.close()
    else:
        print(f"Skipping confusion matrix plot for {dataset_name} due to only one predicted or true class.")


    # Bias group analysis
    df_test= X_test.copy()
    df_test["sex"]= data["sex"].loc[df_test.index]
    df_test["race"] = data["race"].loc[df_test.index]
    df_test["true"] = y_true.values
    df_test["pred"] = preds.values

    acc_by_sex = df_test.groupby("sex").apply(lambda g: (g["true"] == g["pred"]).mean()).to_dict()
    acc_by_race= df_test.groupby("race").apply(lambda g: (g["true"] == g["pred"]).mean()).to_dict()

    bias_results.append({
        "dataset": dataset_name,
        "acc_by_sex": acc_by_sex,
        "acc_by_race": acc_by_race
    })

# Save classification metrics
eval_df = pd.DataFrame(eval_results, columns=["Dataset", "Accuracy", "Precision", "Recall", "F1"])
eval_df.to_csv("evaluation_metrics.csv", index=False)
print("Saved evaluation_metrics.csv")

# Save bias metrics
bias_summary = []
for entry in bias_results:
    for sex, acc in entry["acc_by_sex"].items():
        bias_summary.append({"Dataset": entry["dataset"], "Group": f"sex:{sex}", "Accuracy": acc})
    for race, acc in entry["acc_by_race"].items():
        bias_summary.append({"Dataset": entry["dataset"], "Group": f"race:{race}", "Accuracy": acc})
bias_df = pd.DataFrame(bias_summary)
bias_df.to_csv("bias_metrics.csv", index=False)
print("Saved bias_metrics.csv")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(bias_df)
plt.xticks(rotation=45)
plt.title("Accuracy by Group (sex & race) across Datasets")
plt.tight_layout()
plt.savefig("bias_group_accuracy.png")
plt.show()


# **6.Hyperparameter Tuning**

In this section, we conducted a comprehensive hyperparameter tuning experiment to identify the optimal configuration of learning rate, batch size, architecture, and dropout rate for our feedforward neural network model.

**Objective**

To systematically explore the effect of core hyperparameters on model performance and identify which combination delivers the highest F1-score and generalization on the test data.

We experimented with:

**Learning Rates:** 0.1, 0.01, 0.001

**Batch Sizes:**32, 64, 128

**Architectures:**

  * [64, 32] (shallow)

  * [128, 64, 32] (moderate)

  * [256, 128, 64] (deeper)

* Dropout Rates: 0.0, 0.3, 0.5

This resulted in a total of 81 combinations being evaluated

**Best Configuration**

Learning Rate: 0.001

Batch Size: 64

Architecture: [128, 64, 32]

Dropout: 0.3

F1 Score: 0.735

Accuracy: 0.865

**Observations**

* Models trained with learning rate 0.001 consistently outperformed those using 0.01 or 0.1. A high learning rate often led to unstable training.

* A batch size of 64 provided the best trade-off between convergence speed and generalization.

* The [128, 64, 32] architecture with moderate depth performed the best across most configurations.

* Dropout around 0.3 provided optimal regularization — improving generalization without causing underfitting.

* Very deep architectures or high dropout (> 0.5) often reduced performance.

**Conclusion**

* The best hyperparameters were:
Learning rate 0.001, Batch size 64, Architecture [128, 64, 32], and Dropout 0.3

* This configuration gave the best generalization, and we used it as our final model for subsequent evaluation and analysis tasks.

* Hyperparameter tuning significantly impacted performance and is essential in any deep learning pipeline.



In [ ]:
# Grid values
learning_rates= [0.1, 0.01, 0.001]
batch_sizes= [32, 64, 128]
architectures = {
    "arch_64_32": [64, 32],
    "arch_128_64_32": [128, 64, 32],
    "arch_256_128_64": [256, 128, 64]
}
dropout_rates= [0.0, 0.3, 0.5]

# Dataset to use
split =splits["label_imputed_StandardScaler"]

# Prepare full train and val
X_train = pd.concat([split['X_train'], split['X_val']])
y_train= pd.concat([split['y_train'], split['y_val']])
X_val = split['X_test']
y_val = split['y_test']

if isinstance(y_train, pd.DataFrame):
    y_train = y_train.iloc[:, 0]
if isinstance(y_val, pd.DataFrame):
    y_val = y_val.iloc[:, 0]

X_train_tensor = torch.tensor(X_train.values).float()
y_train_tensor = torch.tensor(y_train.apply(lambda x: 1 if x > 0 else 0).values).float()
X_val_tensor = torch.tensor(X_val.values).float()
y_val_tensor = torch.tensor(y_val.apply(lambda x: 1 if x > 0 else 0).values).int()

# Store results
tuning_results = []

# Training loop
for lr in learning_rates:
    for batch_size in batch_sizes:
        for arch_name, hidden_dims in architectures.items():
            for dropout in dropout_rates:
                print(f"\n Config: lr={lr}, bs={batch_size}, arch={arch_name}, dropout={dropout}")

                # Prepare DataLoader
                train_loader = DataLoader(
                    TensorDataset(X_train_tensor, y_train_tensor),
                    batch_size=batch_size,
                    shuffle=True
                )

                # Model
                model = FeedforwardNN(
                    input_dim=X_train_tensor.shape[1],
                    hidden_dims=hidden_dims,
                    activation_fn=nn.ReLU,
                    use_dropout=True if dropout > 0 else False,
                    use_batchnorm=True
                ).to(device)

                if dropout > 0:
                    for layer in model.model:
                        if isinstance(layer, nn.Dropout):
                            layer.p = dropout

                model.model[-1] = nn.Identity()  # For BCEWithLogitsLoss

                optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-2)
                criterion = nn.BCEWithLogitsLoss()

                # Train
                model, _ = train_model(model, train_loader, (X_val_tensor, y_val_tensor), optimizer, criterion, patience=3, max_epochs=30)

                # Evaluate
                model.eval()
                with torch.no_grad():
                    preds = (model(X_val_tensor.to(device)).view(-1) > 0).int().cpu()
                acc = accuracy_score(y_val_tensor.cpu(), preds)
                f1 = f1_score(y_val_tensor.cpu(), preds)

                tuning_results.append({
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "architecture": arch_name,
                    "dropout": dropout,
                    "accuracy": acc,
                    "f1_score": f1
                })
                print(f" Accuracy: {acc:.4f} | F1: {f1:.4f}")

# Save results
tuning_df = pd.DataFrame(tuning_results)
tuning_df.to_csv("hyperparameter_tuning_results.csv", index=False)
print("\n Saved hyperparameter_tuning_results.csv")


In [ ]:
import seaborn as sns
plt.figure(figsize=(14, 6))
sns.scatterplot(data=tuning_df, x="learning_rate", y="f1_score", hue="architecture", style="dropout", size="batch_size", sizes=(40, 140))
plt.title("F1 Score vs Learning Rate by Architecture & Dropout")
plt.tight_layout()
plt.savefig("hyperparameter_tuning_plot.png")
plt.show()


**Top Configurations**

In [ ]:
# Load the results
tuning_df = pd.read_csv("hyperparameter_tuning_results.csv")

# Sort by F1 score descending
top_configs = tuning_df.sort_values(by="f1_score", ascending=False)

# Display top 5 configurations
top_configs.head(5)


# **7.Explainability**

In this final section, we focus on interpreting the trained model’s decisions using SHAP (SHapley Additive exPlanations), a model-agnostic explainability tool. The goal is to understand which features contribute most to the model's predictions, both at a global and individual level.

**Objective**

* Identify which features most influence income prediction

* Visualize global feature importance

* Explain individual predictions using force plots

* Detect any potential feature-based bias or unexpected behavior

* Compare interpretability of the model with vs. without dropout or batch normalization (optional)

**Method Used: SHAP (KernelExplainer)**

We used SHAP's KernelExplainer, which estimates feature importance by simulating changes to inputs and measuring their effect on model output. This method was chosen because:

It works well with any black-box model (like our feedforward network)

It provides both global and local explanations

We selected 100 random test samples for analysis to reduce computation time.

**Global Feature Importance**

The SHAP summary plot revealed that the following features had the most influence on the model's predictions:

* education-num: Higher educational levels had the strongest positive impact on predicting ">50K"

* capital-gain: Contributed heavily to predicting high-income individuals

* age: Older individuals were more likely to be predicted as high earners

* marital-status and hours-per-week: Also played significant roles

This suggests the model has learned meaningful socioeconomic patterns, similar to what we would expect logically.

**When I looked at the SHAP values, I was relieved to see that features like sex and race didn’t dominate the model’s decisions. They were present, but not nearly as influential as things like education, capital gain, or age. So while there might still be bias from the data, the model itself doesn’t seem to rely too much on sensitive features**

In [ ]:
# Step 1: Define best model (without sigmoid at the end)
model = FeedforwardNN(
    input_dim=X_test_df.shape[1],
    hidden_dims=[128, 64, 32],
    activation_fn=nn.ReLU,
    use_dropout=True,
    use_batchnorm=True
).to(device)
model.model[-1] = nn.Identity()
model.eval()

# Step 2: Define model forward for SHAP
def model_forward(x_numpy):
    x_tensor = torch.tensor(x_numpy, dtype=torch.float32).to(device)
    with torch.no_grad():
        return model(x_tensor).cpu().numpy()

# Step 3: Sample test data
X_sample = X_test_df.sample(100, random_state=42)
X_sample_np = X_sample.values

# Step 4: SHAP Explainer
import shap
shap.initjs()
explainer = shap.KernelExplainer(model_forward, shap.kmeans(X_sample_np, 30))
shap_values = explainer.shap_values(X_sample_np)

# Step 5: Summary Plot
shap.summary_plot(shap_values, X_sample_np, feature_names=X_sample.columns.tolist())


This level of transparency is essential when building models that affect people, like income or hiring predictions.

Through extensive experimentation across 18 dataset variants, multiple loss functions, optimizers, and hyperparameters, I was able to identify the best-performing configuration and gain insights into how the model makes decisions. Tools like SHAP helped me confirm that the model's predictions were largely driven by logical features like education and capital gain, and not overly dependent on sensitive attributes like sex or race. Overall, this project gave me hands-on experience in both engineering robust ML pipelines and evaluating models from a responsible AI lens.

path for all the plots and images: "/content/sample_data"